# Trajectory Curves

## Load Image

In [1]:
import cv2
import numpy as np
import math
import rosbag
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError

In [2]:
# Get segmented images from the rosbag and convert to opencv Images
bag = rosbag.Bag('subset.bag')
bridge = CvBridge()
segmented_images_cv2 = []
for topic, msg, t in bag.read_messages(topics=['/segmented_image']):
    segmented_images_cv2.append(bridge.imgmsg_to_cv2(msg, "bgr8"))

In [3]:
## Get image shape , rows, columns and channels 
shape = segmented_images_cv2[0].shape
height = shape[0]
width = shape[1]
print(len(segmented_images_cv2))

1064


In [4]:
horizon = int(height * 0.6)

In [5]:
def is_red_pixel(image, x, y):
    return int(image[y, x, 2] == 255 and image[y, x, 0] == 0 and image[y, x, 1] == 0)

In [6]:
def center_trajectories(image, r, visualize=True):
    red_pixel_count = 0
    for y in range(height - 1, horizon +1 , -1):
        xL = int((width/2 -r))
        xR = int((width/2 +r))
        for x in range(xL,xR):
            red_pixel_count += is_red_pixel(image, x, y)
            if visualize and int(image[y, x, 2] == 255 and image[y, x, 0] == 0 and image[y, x, 1] == 0):
                image[y,x] = (255,255,255)
            elif int(image[y, x, 2] == 0 and image[y, x, 0] == 0 and image[y, x, 1] == 255):
                return
    return red_pixel_count

In [7]:
## Right Trajectories
def right_trajectories(image, R, r, LTolerance, visualize=True):
    red_pixel_count = 0
    for y in range(height - 1, horizon +1 , -1):
        xL = width
        xR = width
        if (R+r) * (R+r)-(y-height)*(y-height) >= 0 :
            xL = int((width/2+(R-r))-math.sqrt((R+r)*(R+r)-(y-height)*(y-height)))
        if (R-r) * (R-r)-(y-height)*(y-height) >= 0:
            xR = int((width/2+(R+r))-math.sqrt((R-r)*(R-r)-(y-height)*(y-height)))
        xL = max( min(xL,width), 0 )
        xR = max( min(xR,width), 0 )
        x_count = 0
        for x in range(xL, xR):
            red_pixel_count += is_red_pixel(image, x, y)
            if visualize and int(image[y, x, 2] == 255 and image[y, x, 0] == 0 and image[y, x, 1] == 0):
                image[y,x] = (255,255,255)
            elif int(image[y, x, 2] == 0 and image[y, x, 0] == 0 and image[y, x, 1] == 255):
                if (x_count < LTolerance):
                    return
                else:
                    break
            x_count += 1
    return red_pixel_count

In [8]:
## Left Trajectories
def left_trajectories(image, R, r, LTolerance, visualize=True):
    red_pixel_count = 0
    #image_copy = image.copy()
    for y in range(height - 1, horizon +1 , -1):
        xL = 0
        xR = 0
        if (R-r)*(R-r)-(y-height)*(y-height) >= 0 :
            xL = int((width/2-(R+r))+math.sqrt((R-r)*(R-r)-(y-height)*(y-height)))
        if (R+r)*(R+r)-(y-height)*(y-height) >= 0:
            xR = int((width/2-(R-r))+math.sqrt((R+r)*(R+r)-(y-height)*(y-height)))
        xL = max(min(xL,width),0)
        xR = max(min(xR,width),0)
        x_count = 0
        for x in range(xR - 1, xL, -1):
            red_pixel_count += is_red_pixel(image, x, y)
            if visualize and int(image[y, x, 2] == 255 and image[y, x, 0] == 0 and image[y, x, 1] == 0):
                image[y,x] = (255,255,255)
            elif int(image[y, x, 2] == 0 and image[y, x, 0] == 0 and image[y, x, 1] == 255):
                if (x_count < LTolerance):
                    return
                else:
                    break
            x_count += 1
    return red_pixel_count

In [25]:
print left_trajectories(blank_image,200, 10, 10, False)
print center_trajectories(blank_image,20, 10, False)
print right_trajectories(blank_image,200, 10, 10, False)

6398
7005
6135


In [23]:
left_trajectories(segmented_images_cv2[100],100, 10, 10)
left_trajectories(segmented_images_cv2[100],150, 10, 10)
left_trajectories(segmented_images_cv2[100],200, 10, 10)
center_trajectories(segmented_images_cv2[100],20)
right_trajectories(segmented_images_cv2[100],200, 10, 10)
right_trajectories(segmented_images_cv2[100],150, 10, 10)
right_trajectories(segmented_images_cv2[100],100, 10, 10)

In [10]:
for idx, image in enumerate(segmented_images_cv2):
    image_copy = image.copy()
    left_trajectories(image_copy,100, 10, 10)
    left_trajectories(image_copy,150, 10, 10)
    left_trajectories(image_copy,200, 10, 10)
    center_trajectories(image_copy,20)
    right_trajectories(image_copy,200, 10, 10)
    right_trajectories(image_copy,150, 10, 10)
    right_trajectories(image_copy,100, 10, 10)
    cv2.imwrite('traject/{}_traject.jpg'.format(idx), image_copy)

In [24]:
# cv2.imshow('image',segmented_images_cv2[100])
# cv2.waitKey(0)
# cv2.destroyAllWindows()